In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np  
import folium
import sqlite3

# Introduction

This project has 2 major parts:
- Comprehensive EDA
- Statistical + ML modeling

The goal is to predict future restaurant visitor counts using a real-world dataset provided by Recruit Holdings for a 2018 data science competition. All analysis and modeling are performed in Python, utilizing widely used data science and machine learning libraries (see the requirements file for details).

This is a time-series forecasting problem, with data collected from a diverse range of Japanese restaurants across the country. These restaurants use either AirREGI (air), a reservation and point-of-sale system or Hot Pepper Gourmet (HPG), a platform similar to Yelp for restaurant discovery and reservations.

The dataset includes eight separate CSV files, covering January 2016 through the third week of April 2017. The sample_submission.csv file is an example of how the results/prediction should be submitted, which spans the last week of April 2017 and all of May 2017 which in Japan is the largest week long holiday called `Golden week`.

A more comprehensive description of each file:

- air_reserve.csv / hpg_reserve.csv : Historical visit data for the air and hpg restaurants. It includes:
    - air_store_id / hpg_store_id
    - visit_datetime: Time of visit after making a reservation
    - reserve_datetime: Time which the reservation was made
    - reserve_visitors: Number of reservations made

- air_visit_data.csv (Only for air stores)
    - air_store_id 
    - visit_date
    - visitors: Total number of visitors (both reservations + no reservations) for Air stores

- air_store_info.csv / hpg_store_info.csv
    - air_store_id / hpg_store_id
    - air_genre_name / hpg_genre_name: Cuisine type
    - air_area_name / hpg_area_name: Area in which the store is located
    - latitude
    - longitude

- date_info.csv
    - calendar_date
    - day_of_week
    - holiday_flg: 1 for holiday, 0 otherwise

- store_id_relation.csv (Some stores are registered with both air and hpg)
    - air_store_id
    - hpg_store_id

- sample_submission.csv
    - id: air store id combined with date
    - visitors

# Loading data

Let's load all the CSV files and automatically convert any columns containing date or time information from object type to proper datetime format. This ensures that all temporal features are ready for analysis and modeling.

In [ ]:
air_reserve = pd.read_csv('../data/raw/air_reserve.csv')
air_store_info = pd.read_csv('../data/raw/air_store_info.csv')
air_visit_data = pd.read_csv('../data/raw/air_visit_data.csv')
date_info = pd.read_csv('../data/raw/date_info.csv')
hpg_reserve = pd.read_csv('../data/raw/hpg_reserve.csv')
hpg_store_info = pd.read_csv('../data/raw/hpg_store_info.csv')
sample_submission = pd.read_csv('../data/raw/sample_submission.csv')
hpg_air_store_id_relation = pd.read_csv('../data/raw/store_id_relation.csv')

datasets = {
    'air_reserve': air_reserve,
    'air_store_info': air_store_info,
    'air_visit_data': air_visit_data,
    'date_info': date_info,
    'hpg_reserve': hpg_reserve,
    'hpg_store_info': hpg_store_info,
    'sample_submission': sample_submission,
    'hpg_air_store_id_relation': hpg_air_store_id_relation
}

# convert datetime object columns to datetime
for name, df in datasets.items():
    for col in df.columns:
        if df[col].dtype == "object" and ("date" in col.lower() or "time" in col.lower()):
            df[col] = pd.to_datetime(df[col], errors="coerce")


In [4]:
# Display information for each dataset
for name, df in datasets.items():
    print(f'{name} Data Info:'.upper())
    print(df.info())
    print('\n')

AIR_RESERVE DATA INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92378 entries, 0 to 92377
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   air_store_id      92378 non-null  object        
 1   visit_datetime    92378 non-null  datetime64[ns]
 2   reserve_datetime  92378 non-null  datetime64[ns]
 3   reserve_visitors  92378 non-null  int64         
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 2.8+ MB
None


AIR_STORE_INFO DATA INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   air_store_id    829 non-null    object 
 1   air_genre_name  829 non-null    object 
 2   air_area_name   829 non-null    object 
 3   latitude        829 non-null    float64
 4   longitude       829 non-null    float64
dtypes: float64(2), obje

In [5]:
# Display descriptive statistics for each dataset
for name, df in datasets.items():
    print(f'{name} Data Description:'.upper())
    print(df.describe(include='all'))
    print('\n')

AIR_RESERVE DATA DESCRIPTION:
                air_store_id                 visit_datetime  \
count                  92378                          92378   
unique                   314                            NaN   
top     air_8093d0b565e9dbdf                            NaN   
freq                    2263                            NaN   
mean                     NaN  2016-12-05 08:18:58.751651072   
min                      NaN            2016-01-01 19:00:00   
25%                      NaN            2016-11-15 19:00:00   
50%                      NaN            2017-01-05 18:00:00   
75%                      NaN            2017-03-03 19:00:00   
max                      NaN            2017-05-31 21:00:00   
std                      NaN                            NaN   

                     reserve_datetime  reserve_visitors  
count                           92378      92378.000000  
unique                            NaN               NaN  
top                               NaN  

# EDA

## Univariate feature visualizations

### Air Visits

![Daily Total Visitors](../reports/figures/air_visits/daily_total_visitors.png)

Daily total visitors exhibit strong weekly seasonality, with clear peaks likely corresponding to weekends. Visitor counts show a significant increase around mid-2016, maintaining an upward trend thereafter. This significant increase could be due to the counts being summed together from both 2016 and 2017. Occasional sharp drops, particularly in early 2017, may reflect holidays, data gaps, or external disruptions or just shop closures.

![Year-over-year](../reports/figures/air_visits/year_over_year_visitor_trends.png)

The year-over-year visitor trends show strong seasonal patterns with clear weekly peaks and troughs. Visitor counts in 2017 are consistently higher than in 2016, indicating overall growth in customer traffic. Notable spikes in late December and early spring suggest holiday and seasonal event impacts.

![Histogram of visitors (log scale)](../reports/figures/air_visits/histogram_visitors_log.png)

The log scaled histogram of visitors shows a heavily right skewed distribution. Most restaurants receive fewer than 10 daily visitors, with a long tail of high volume outliers. This suggests significant variance in restaurant traffic, highlighting the need for log transformation or robust models to handle skewness.

![Median visitors by Weekday](../reports/figures/air_visits/median_visitors_by_weekday.png)

Saturdays can be expected to be the busiest days closely followed by Friday and Sunday. Perhaps due to the weekend rush. Whereas, Mondays and Tuesdays can expect fewer visitors

![Median visitors by Month](../reports/figures/air_visits/median_visitors_by_month.png)

Visitor levels are relatively stable between March and June, but dips slightly in August and September, and peak sharply in December, likely due to holiday-related dining activity.

![Top 5](../reports/figures/air_visits/top5_total_visitors_by_air_store.png)

The top 5 stores account for a significant share of total visitors, each attracting approximately 18,000–19,000 visitors.

![Bottom 5](../reports/figures/air_visits/bottom5_total_visitors_by_air_store.png)

The bottom 5 stores have significantly lower visitor counts, ranging from 200 to 800 total visitors. This sharp contrast with top-performing stores suggests uneven demand across locations, potentially due to store visibility, competition or location.



### Air Reservations

![Total Daily Reservations](../reports/figures/air_reserve/daily_total_reservations.png)

 Visitor counts start low in early 2016, showing modest and steady growth, but there is a noticeable gap with zero activity around mid-2016, likely due to missing data rather than an operational halt. From October 2016 onward, visitor counts surge sharply, peaking above 4,000 daily, with clear weekly fluctuations likely reflecting weekend peaks and seasonal demand. Early 2017 continues to show strong activity with increased volatility before declining sharply around March, coinciding with the dataset’s end

![Hourly total reservation](../reports/figures/air_reserve/hourly_total_reservations.png)

Reservations are minimal during early morning hours (2:00–7:00) but start to increase gradually from late morning onward. A significant surge begins around 11:00, with steady growth through the afternoon. Peak activity occurs between 17:00 and 18:00, where reservation counts exceed 55,000, indicating a strong preference for evening dining or late-day bookings. After 18:00, reservations decline steadily but remain relatively high through 22:00, reflecting typical restaurant operating hours.

![Monthly total reservation](../reports/figures/air_reserve/monthly_total_reservations.png)

Reservations are highest in March, peaking at over 75,000, followed by strong activity in January, February, November, and December, all exceeding 60,000 reservations. A significant decline occurs from May onwards, with the summer months (June–September) showing minimal reservation activity, particularly in August, which records almost no reservations. The recovery begins in October and accelerates into the holiday season in November and December. This pattern suggests strong seasonal demand, with peaks in early spring and late year, likely influenced by holidays and seasonal dining trends.

![Visit time since reservation](../reports/figures/air_reserve/visit_time_distribution.png)

The data is highly right skewed, with the vast majority of reservations made very close to the visit date—often on the same day or within a few days. The frequency drops sharply as the time gap increases, with very few reservations booked more than a month in advance. This pattern suggests that most customers prefer to make last-minute or short-term reservations rather than planning far ahead.

### HPG Reservations

![Daily total reservations](../reports/figures/hpg_reserve/daily_total_reservations.png)

This line chart shows the daily total reservations made through the HPG platform over time. Reservations display a clear weekly pattern, with recurring peaks that suggest higher bookings on specific days of the week. There is a noticeable increase in reservation volume toward the end of 2016, peaking sharply around late December, likely due to the holiday season. After this peak, reservations remain relatively high in early 2017 but gradually decline, dropping significantly around May 2017. This trend indicates strong seasonality in reservations, with notable spikes during holiday periods and a sharp overall drop-off afterward.

![Hourly total reservations](../reports/figures/hpg_reserve/hourly_total_reservations.png)

The data reveals two distinct peaks: one around lunchtime at 12 PM, and a much larger one during evening hours, particularly between 6 PM (18:00) and 9 PM (21:00), with 6 PM marking the highest reservation activity. Early morning hours (0–7 AM) have very low reservation volumes, indicating minimal activity overnight. This pattern reflects typical dining behaviors, where most reservations are concentrated around lunch and especially dinner times.

![Monthly total reservations](../reports/figures/hpg_reserve/monthly_total_reservations.png)

March stands out as the peak month with the highest reservation volume, followed by strong activity in December and January. There is a clear seasonal decline from May through September, with these months recording the lowest reservation counts. Activity then begins to rise again in October, likely due to holiday seasons and year-end gatherings

![Visit time since reservation](../reports/figures/hpg_reserve/visit_time_distribution.png)

The majority of reservations are made very close to the visit date, with a sharp peak at 0–2 days, indicating a strong tendency for last-minute bookings. The frequency of reservations declines rapidly as the lead time increases, with very few bookings made beyond 30 days in advance

### Air Store

![air map](../reports/figures/air_store_info/air_map.png)

The locations are widely spread, with significant clusters around major urban areas such as Tokyo, Osaka, Fukuoka, and Hiroshima, indicating strong restaurant representation in metropolitan regions. Smaller clusters appear in other parts of the country, suggesting coverage in both highly populated cities and regional areas.

![Number of Stores of Each Genre air](../reports/figures/air_store_info/store_of_genre.png')

Izakaya leads with the highest count, closely followed by Cafés/Sweets, indicating a strong demand for casual dining and dessert spots. Dining bars and Italian/French cuisines are also well-represented, reflecting popularity in Western-style dining. Bar/Cocktail and Japanese food hold moderate representation, while genres like Yakiniku/Korean food, Okonomiyaki/Monja/Teppanyaki, and Creative cuisine have smaller counts. Niche categories such as International cuisine, Asian, and Karaoke/Party have very few establishments, suggesting limited but specialized offerings. Overall, the chart highlights a market dominated by casual and social dining experiences.

![Number of Air Restaurants per Perfecture](../reports/figures/air_store_info/air_restaurants_per_perfecture.png)

Tokyo-to overwhelmingly dominates with over 400 restaurants, highlighting its position as the primary hub for dining establishments. Fukuoka-ken follows at a distant second, with a little over 120 restaurants, while Osaka-fu holds the third spot with around 70. Other prefectures like Hyōgo-ken, Hokkaidō, and Hiroshima-ken have moderate representation, whereas Shizuoka-ken, Miyagi-ken, and Niigata-ken have relatively few restaurants

### HPG Stores


![air map](../reports/figures/hpg_store_info/map.png)

The markers indicate that restaurants are primarily concentrated in urban and economically significant regions, with major clusters around Tokyo, Osaka, and Kyushu. The Tokyo metropolitan area has the densest grouping, consistent with its high population density and tourism demand. Other noticeable clusters appear in Kansai (Osaka-Kobe-Kyoto area) and parts of southern Japan, while northern and rural regions have fewer establishments.

![Number of Stores of Each Genre](../reports/figures/hpg_store_info/store_of_genre_hpg.png')

The most dominant category is Japanese-style cuisine, with nearly 1,800 stores, reflecting the strong cultural and consumer preference for traditional Japanese food. Other notable categories include International cuisine (~700 stores), Grilled meat, and Creative cuisine, each showing significant representation. Meanwhile, genres such as French, Spanish, and Dim Sum/Dumplings have only a small presence, indicating niche but specialized offerings. Overall, the chart highlights that HPG restaurants are heavily skewed toward Japanese and Asian cuisine, while Western and other international genres make up a smaller proportion of the total. This suggests a market largely driven by local tastes with some diversification toward global food trends.


![Number of Air Restaurants per Perfecture](../reports/figures/hpg_store_info/hpg_restaurants_per_perfecture.png)

Tokyo dominates overwhelmingly, with over 2,000 restaurants, highlighting its position as the country’s largest metropolitan hub and a center for dining and tourism. Osaka follows distantly with just above 500 restaurants, while other prefectures such as Fukuoka, Hyōgo, and Hokkaidō have between 300–400 restaurants each. The remaining prefectures, including Shizuoka, Hiroshima, Niigata, and Miyagi, show much smaller counts, under 250 restaurants each. Interestingly, there is also a “None” category, which likely represents records with missing location data. Overall, the distribution indicates that HPG’s coverage is highly concentrated in Tokyo and major urban areas, while regional prefectures are much less represented.

### Store ID Relation

![Stores that uses both Air and HPG](../reports/figures/Screenshot%202025-08-26%20at%2001.03.30.png)

Similar to the Air dataset, the restaurants are concentrated in major metropolitan areas, with a strong cluster around Tokyo and the Kanto region, as well as notable concentrations in the Kansai region (Osaka, Kyoto, Kobe) and Fukuoka in Kyushu. Smaller clusters appear in northern Japan (Tohoku and Hokkaido) and scattered areas along Honshu’s western coast. Compared to the Air dataset, HPG restaurants appear to have broader coverage across Japan, but still follow the same urban-centric trend, reflecting where population density and dining demand are highest. This reinforces the idea that restaurant availability and reservation activity are heavily influenced by urbanization and regional economic hubs.

### Holidays

![Holidays over time](../reports/figures/date_info/holiday_flag_over_time.png)

The vertical spikes at 1 represent holidays across 2016 and early 2017. We can see that holidays are relatively evenly distributed throughout the year, with higher clustering in autumn and around New Year (late December to early January). These holiday periods are important because they typically lead to spikes in restaurant visits and reservations, reflecting cultural and seasonal dining patterns in Japan

![Holidays by month](../reports/figures/date_info/holiday_distribution_by_month.png)

January stands out with the highest number of holidays (8), which is linked to New Year celebrations in Japan, a period of major travel and dining activity. May is another peak with 6 holidays, largely due to Golden Week, a cluster of consecutive national holidays that drives a sharp rise in restaurant visits. July, August, and December each have 4 holidays, aligning with summer festivals, Obon season, and year-end festivities, respectively. By contrast, June and September–October have the fewest holidays, reflecting quieter months for holiday-driven restaurant activity.

![Holidays by day of week](../reports/figures/date_info/holiday_distribution_by_day_of_week.png)

The clear standout is Monday, with the highest count (9 holidays), followed by Thursday (7 holidays). This pattern suggests that holidays in Japan often extend weekends or create “long weekends,” especially when aligned with Mondays or Thursdays. Fridays and Saturdays also host a moderate number of holidays (5 each), while Sundays (4) are less common, since they are already a non-working day. On the other end, Tuesdays (2 holidays) and Wednesdays (3 holidays) see the fewest, making midweek holidays relatively rare.

## Multivariate Feature Visualizations

![Food cat vs day of week](../reports/figures/feature_relations/genre_vs_dayofweek_heatmap_reservations.png)

The standout trend is that Izakaya restaurants dominate, especially on Fridays and Saturdays, where the intensity peaks (deep blue cells). This suggests that Izakayas are a go-to choice for weekend socializing and gatherings.

Another strong genre is Italian/French, which shows consistent reservations across the week, with higher intensity on Fridays and Saturdays as well—though not as concentrated as Izakayas. Japanese cuisine/Kaiseki and Dining bars also show weekend boosts, while most other genres (like cafes, grilled meat, or Western food) remain relatively low and steady throughout the week.

The weekly pattern is clear: weekend days (Friday–Saturday, sometimes Sunday) see the largest spikes in reservations across almost all food genres, with weekdays (Monday–Thursday) being calmer. This reflects cultural dining behavior in Japan, where people tend to go out more for leisure eating and social drinking toward the end of the week.

![food cat vs week](../reports/figures/feature_relations/food_category_vs_week_heatmap.png)

The most striking observation is the dominance of Izakaya restaurants, which consistently record the highest reservation volumes, especially from late 2016 through early 2017, peaking at more than 1,400 weekly visits. Italian/French restaurants also stand out with sustained high demand during the same period, though at a slightly lower intensity than Izakaya.

Other categories such as Japanese cuisine/Kaiseki, Dining bars, and Western food show moderate weekly activity, but their intensity remains much lower in comparison. Most other genres—including cafés, creative cuisine, and bars—register very low and sporadic activity across weeks, highlighted by their faint shading in the heatmap.

Overall, the chart reveals that a few dominant food genres (Izakaya, Italian/French) drive the bulk of reservations, especially in the latter part of the dataset, while most other genres contribute minimally

![comb visits vs holidays](../reports/figures/feature_relations/visitors_vs_holiday_flag.png)

The boxplot shows that restaurants tend to have slightly more visitors on holidays compared to non-holidays, with the median number of visitors being higher and the variability much larger on holidays. While both distributions overlap considerably, holidays are associated with more extreme spikes, as seen in the numerous high outliers. 

![](../reports/figures/feature_relations/food_category_vs_area_heatmap.png)

The brighter clusters indicate higher concentrations of restaurants in specific food genres within certain regions. Notably, Izakaya and Japanese-style cuisine stand out with dense concentrations across many areas, especially in major urban centers like Tokyo and Osaka. Other categories, such as Italian/French, Café/Sweets, and Dining bars, are also widely spread but with smaller concentrations. Meanwhile, more niche cuisines (e.g., Thai/Vietnamese, Sichuan, or Spanish/Mediterranean) appear only sporadically across regions